In [2]:
COLLECTION_REVIEW = 'amzn-reviews'
COLLECTION_PRODUCT = 'amzn-products'
CONFIG_REVIEW = 'C:\\Users\\M_M\\Desktop\\assignment2\\conf\\amzn-reviews'
CONFIG_PRODUCT = 'C:\\Users\\M_M\\Desktop\\assignment2\\conf\\amzn-products'
    
create_collection(CONFIG_REVIEW, COLLECTION_REVIEW)
create_collection(CONFIG_PRODUCT, COLLECTION_PRODUCT)

In [3]:
def product_json(filename):
    list_of_dictionaries = []
    with open(filename, "r") as f:
        data = f.readlines()
        for line in data:
            newLine = eval('(' + line + ')')
            title = newLine.get('title')
            if  title is not None and len(title) > 0:
                product = {
                    'productID': newLine.get('asin'),
                    'description': newLine.get('description'),
                    'productName': title,
                    'price': newLine.get('price')
                }
                list_of_dictionaries.append(product)
    return list_of_dictionaries

def review_json(filename, products):
    list_of_dictionaries = []
    set_p = set()
    for product in products:
        set_p.add(product['productID'])
    with open(filename, "r") as f:
        data = f.readlines()
        for line in data:
            newLine = eval('(' + line + ')')
            asin = newLine.get('asin')
            if asin in set_p:
                reviews = {
                    'productID': asin,
                    'reviewerName': newLine.get('reviewerName'),                      
                    'reviewScore': int(newLine.get('overall')),  
                    'reviewSummary': newLine.get('summary'),
                    'reviewBody': newLine.get('reviewText')
                }
                list_of_dictionaries.append(reviews) 
    return list_of_dictionaries

In [4]:
import pysolr
import json
import os 

solr_p = pysolr.Solr('http://localhost:8983/solr/amzn-products')
solr_r = pysolr.Solr('http://localhost:8983/solr/amzn-reviews')
def index_documents(product_filename, review_filename):
    productList = product_json(product_filename)
    reviewList = review_json(review_filename, productList)
    docs_p = []
    docs_r = []
    for product in productList:
        docs_p.append(json.loads(json.dumps(product)))
    solr_p.add(docs_p, commit=True)  
    for review in reviewList:
        docs_r.append(json.loads(json.dumps(review)))      
    solr_r.add(docs_r, commit=True)      

#### Instruction:
    for the solr: 
        we are using the solr-8.1.0
    for the flask:
        Go into the directory "./assignment2/reviewsite"
        create venv and activate it: >py -3 -m venv venv
        activate venv: ./reviewsite>venv\Scripts\activate
        please install: 
        (venv)./reviewsite>pip install flask_wtf
        (venv)./reviewsite>pip install requests
        Then run:
        (venv)./reviewsite>set FLASK_APP=__init__.py
        (venv)./reviewsite>flask run